### Use rio-viz in Jupyter Notebook

In [9]:
import time
import httpx

from ipyleaflet import Map, ScaleControl, FullScreenControl, SplitMapControl, TileLayer

from rio_tiler.io import STACReader
from rio_viz.app import Client

In [10]:
# Create rio-viz Client (using server-thread to launch backgroud task)
client = Client(
    "https://earth-search.aws.element84.com/v0/collections/sentinel-s2-l2a-cogs/items/S2A_34SGA_20200318_0_L2A",
    reader=STACReader,
    # By default STACReader min/max zoom is 0->24
    # Knowledge of the Sentinel-2 data tell us it's more 8->14
    minzoom=8,
    maxzoom=14,
)

# Gives some time for the server to setup
time.sleep(1)

# Check that client is running
print("Client is alive: ", client.server.is_alive())

Client is alive:  True


In [11]:
# Fetch info
httpx.get(f"{client.endpoint}/info").json()

{'overview': {'bounds': [699960.0, 3490240.0, 809720.0, 3600000.0],
  'crs': 'http://www.opengis.net/def/crs/EPSG/0/32634',
  'band_metadata': [['b1', {}], ['b2', {}], ['b3', {}]],
  'band_descriptions': [['b1', ''], ['b2', ''], ['b3', '']],
  'dtype': 'uint8',
  'nodata_type': 'Nodata',
  'colorinterp': ['red', 'green', 'blue'],
  'scales': [1.0, 1.0, 1.0],
  'offsets': [0.0, 0.0, 0.0],
  'driver': 'GTiff',
  'count': 3,
  'width': 343,
  'height': 343,
  'overviews': [2, 4],
  'nodata_value': 0.0},
 'B11': {'bounds': [699960.0, 3490200.0, 809760.0, 3600000.0],
  'crs': 'http://www.opengis.net/def/crs/EPSG/0/32634',
  'band_metadata': [['b1', {}]],
  'band_descriptions': [['b1', '']],
  'dtype': 'uint16',
  'nodata_type': 'Nodata',
  'colorinterp': ['gray'],
  'scales': [1.0],
  'offsets': [0.0],
  'driver': 'GTiff',
  'count': 1,
  'width': 5490,
  'height': 5490,
  'overviews': [2, 4, 8],
  'nodata_value': 0.0},
 'B01': {'bounds': [699960.0, 3490200.0, 809760.0, 3600000.0],
  'crs':

In [14]:
tilejson = httpx.get(
    f"{client.endpoint}/tilejson.json",
    params = {
        "assets": ["B04", "B03", "B02"],
        "rescale": "0,10000"
    }
).json()

bounds = tilejson["bounds"]

layer = TileLayer(
    url=tilejson["tiles"][0],
    min_zoom=tilejson["minzoom"],
    max_zoom=tilejson["maxzoom"],
    bounds=((bounds[0],bounds[2]), (bounds[1], bounds[3])),
)


# Make the ipyleaflet map
m = Map(center=((bounds[1] + bounds[3]) / 2, (bounds[0] + bounds[2]) / 2), zoom=client.minzoom)
m.add_layer(layer)
m

Map(center=[32.00833055925221, 23.794854319372455], controls=(ZoomControl(options=['position', 'zoom_in_text',…

In [13]:
left_tilejson = httpx.get(
    f"{client.endpoint}/tilejson.json",
    params = {
        "assets": ["B04", "B03", "B02"],
        "rescale": "0,10000"
    }
).json()

right_tilejson = httpx.get(
    f"{client.endpoint}/tilejson.json",
    params = {
        "assets": ["B05", "B04", "B03"],
        "rescale": "0,10000"
    }
).json()

bounds = tilejson["bounds"]

left = TileLayer(
    url=left_tilejson["tiles"][0],
    min_zoom=left_tilejson["minzoom"],
    max_zoom=left_tilejson["maxzoom"],
    bounds=((bounds[0],bounds[2]), (bounds[1], bounds[3])),
)

right = TileLayer(
    url=right_tilejson["tiles"][0],
    min_zoom=right_tilejson["minzoom"],
    max_zoom=right_tilejson["maxzoom"],
    bounds=((bounds[0],bounds[2]), (bounds[1], bounds[3])),
)

# Make the ipyleaflet map
m = Map(center=((bounds[1] + bounds[3]) / 2, (bounds[0] + bounds[2]) / 2), zoom=client.minzoom)
control = SplitMapControl(left_layer=left, right_layer=right)

m.add_control(control)
m.add_control(ScaleControl(position='bottomleft'))
m.add_control(FullScreenControl())
m

Map(center=[32.00833055925221, 23.794854319372455], controls=(ZoomControl(options=['position', 'zoom_in_text',…

In [ ]:
client.shutdown()